In [1]:
from database.GraphDB import KnowledgeGraphDB
from retrieval.retrieval_hub import RetrivalHub
from query_and_ranking.query import Query
from query_and_ranking.rerank import Reranker
from agentic import Agent
from tqdm import tqdm

2024-06-09 02:00:36.245799: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-09 02:00:36.245823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-09 02:00:36.246432: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-09 02:00:36.828192: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()

neo4j_uri = os.getenv('NEO4J_URI')
neo4j_user = os.getenv('NEO4J_USER')
neo4j_password = os.getenv('NEO4J_PASSWORD')

In [2]:
kg = KnowledgeGraphDB(uri=neo4j_uri, user=neo4j_user, password=neo4j_password)
retrieval_hub = RetrivalHub(kg)
reranker = Reranker()
cv_query = Query(kg, retrieval_hub)

In [3]:
chat_agent = Agent(kg, retrieval_hub, reranker, cv_query)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
import json
questions = json.loads(open('data/benchmark/qa_cv.json').read())

In [5]:
answers = []
i = 0

for question in tqdm(questions):
    prompt = question['prompt']
    answer = dict()
    answer['prompt'] = prompt
    answer['answer'] = question['answer']
    try:
        res = chat_agent.get_cv_from_jd(prompt)
        ans = res['files']
        answer['predict'] = [a.replace('data/raw/data/clean4.0/', '') for a in ans]
    except:
        answer['predict'] = []
    answers.append(answer)
    with open('data/benchmark/qa_cv_kg.json', 'w') as f:
        f.write(json.dumps(answers))
    i += 1


  0%|          | 0/39 [00:00<?, ?it/s]

Extracting Job Description


You are not running the flash-attention implementation, expect numerical differences.
/home/quanghung20gg/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Languages:\n- English\nMajors:\n- bachelor, computer science, national economics university hanoi, (GPA: 3.43)\nWork:\n- lts group (0.5 years)\n- viettel telecom (0.5 years)\nRoles:\n- software developer (0.5 years)\nSkills:\n- web development\n- api development\n- content management system\n- user interface design\n- artificial intelligence\n- database management\n- testing\n- deployment\n- english teaching\nProgramming languages:\n- java (0.5 years)\n- javascript (0.5 years)\n- typescript (0.5 years)\n- php (0.5 years)\n- spring boot (0.5 years)\n- react (0.5 years)\n- node (0.5 years)\n- graphql (0.5 years)\nCities:\n- ha noi\n', metadata={'id': 1593}), -208.47142856414308), (Document(page_content='Languages:\n- English\nM

End LLM
New Query:  {'cityList': ['ha noi'], 'eduList': ['master', 'undergraduate', 'phd', 'diploma', 'certificate', 'bachelor'], 'roleList': ['marketing manager', 'marketing analyst', 'marketing assistant', 'marketing officer']}
Extended Num Application:  15
Not enough application
New Query:  {'cityList': ['ha noi'], 'roleList': ['marketing manager', 'marketing assistant', 'marketing analyst', 'marketing', 'marketing officer']}
[786, 1371, 1513, 1648, 1802, 2109, 424]
Less -> Sematic Search
Using Sematic Search
Num CV ok:  12
Rerank
[1240, 1371, 1488, 1513, 1593, 1648, 1802, 2004, 2109, 222, 424, 786]
Time: 10.246462345123291
Extracting Job Description


/home/quanghung20gg/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Languages:\n- English\nMajors:\n- undergraduate, information technology, ho chi minh city open university, (GPA: 3.6)\nWork:\n- m q -h o use (1.5 years)\n- healthy heart project (0.5 years)\nRoles:\n- mobile app developer (0.5 years)\n- web developer (0 years)\n- software developer (0 years)\nSkills:\n- web development\n- mobile app development\n- customer service\n- marketing\n- database design\n- api development\nProgramming languages:\n- java (0 years)\n- javascript (0 years)\n- python (0 years)\n- spring mvc (0 years)\n- react (0 years)\n- react native (0 years)\n- mysql (0 years)\n- html (0 years)\n- css (0 years)\nCities:\n- ho chi minh\n', metadata={'id': 1240}), -193.27409230280102), (Document(page_content='Languages:\n- English\nMajors:\n- bachelor, information security, ho chi minh city university of

...

End LLM
New Query:  {'institutionList': ['ho chi minh city university of industry and trade', 'ho chi minh city university of technology and food industry'], 'eduList': ['master', 'undergraduate', 'phd', 'diploma', 'certificate', 'bachelor']}
Extended Num Application:  3
Num CV ok:  3
Rerank
Reranking
[1841, 178]
Time: 9.458472728729248
Extracting Job Description


/home/quanghung20gg/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Languages:\n- English\n- Vietnamese\nMajors:\n- undergraduate, computer networking and communication, ho chi minh city university of technology, (GPA: 3.6)\nWork:\n- hung phat usa (0.5 years)\n- private supermarket (0.5 years)\n- mwg (0.5 years)\n- etomil milk (0.5 years)\n- ho chi minh city university of technology (6.5 years)\nRoles:\n- it support (0.5 years)\n- inventory management staff (0.5 years)\n- security camera supervisor (0.5 years)\n- marketing staff (0.5 years)\n- programmer (0.5 years)\nSkills:\n- system configuration\n- management\n- network administration\n- project management\n- web development\n- mobile development\n- game development\n- security\n- iot\nProgramming languages:\n- html (0.5 years)\n- css (0.5 years)\n- unity (0.5 years)\n- wordpress (0.5 years)\n- sql (0.5 years)\nCities:\n- t

End LLM
New Query:  {'institutionList': ['ho chi minh city university of economics'], 'eduList': ['master', 'undergraduate', 'phd', 'diploma', 'certificate', 'bachelor']}
Extended Num Application:  11
Not enough application
New Query:  {'institutionList': ['ho chi minh city university of economics', 'ho chi minh city university of science']}
[1253, 1428, 669, 468, 135, 1958]
Less -> Sematic Search
Using Sematic Search
Num CV ok:  8
Rerank
[1253, 135, 1428, 1488, 1530, 1958, 468, 669]
Time: 10.224083423614502
Extracting Job Description


/home/quanghung20gg/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Languages:\n- English\nMajors:\n- bachelor, software engineer, hanoi university of industry, (GPA: 3.5)\nRoles:\n- android developer (0 years)\nSkills:\n- user interface design\n- api design\n- firebase\n- android development\n- git\n- postman\n- mvvm architecture\n- clean architecture\nProgramming languages:\n- kotlin (0 years)\n- dart (0 years)\nCities:\n- ha noi\nAwards:\n- Second Prize in Japanese dubbing contest\n- Consolation Prize in School-level Olympic Programming Contest\n- Third Prize in School-level Olympic Programming Contest\n', metadata={'id': 651}), -160.3873319785448), (Document(page_content='Languages:\n- English\nMajors:\n- bachelor, computer science, national economics university hanoi, (GPA: 3.43)\nWork:\n- lts group (0.5 years)\n- viettel telecom (0.5 years)\nRoles:\n- software developer 

End LLM
New Query:  {'institutionList': ['posts and telecommunications institute of technology', 'vnu-hcm university of science', 'fpt polytechnic hà nội', 'university of technology information - vnu', 'university of information technology'], 'eduList': ['master', 'undergraduate', 'phd', 'diploma', 'certificate', 'bachelor']}
Extended Num Application:  12
Not enough application
New Query:  {'institutionList': ['posts and telecommunications institute of technology', 'vnu-hcm university of science', 'fpt polytechnic hà nội', 'university of technology information - vnu', 'university of information technology']}
[363, 751, 1488, 2006, 2051]
Less -> Sematic Search
Using Sematic Search
Num CV ok:  9
Rerank
[1488, 1530, 1593, 19, 2006, 2051, 363, 651, 751]
Time: 8.397864580154419


In [7]:
recall = 0
precision = 0
long_recall = 0
long_precision = 0
len_long = 0
short_recall = 0
short_precision = 0
len_short = 0

for answer in answers:
    num_ans = len(answer['answer'])
    correct = 0
    ans = set(answer['answer'])
    for p in answer['predict']:
        if p in ans:
            correct += 1
    recall += correct / num_ans
    precision += correct / (len(answer['predict'])+1e-5)
    if num_ans > 7:
        long_recall += correct / num_ans
        long_precision += correct / (len(answer['predict'])+1e-5)
        len_long+= 1
    else:
        short_recall += correct / num_ans
        short_precision += correct / (len(answer['predict'])+1e-5)
        len_short += 1

In [8]:
recall /= len(answers)
precision /= len(answers)
long_recall /= len_long
long_precision /= len_long
short_recall /= len_short
short_precision /= len_short
f1 = 2 * recall * precision / (recall + precision + 1e-5)
short_f1 = 2 * short_recall * short_precision / (short_recall + short_precision + 1e-5)
long_f1 = 2 * long_recall * long_precision / (long_recall + long_precision+1e-5)

In [9]:
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1: {f1}")
print("=====================================")

print(f"Short Recall: {short_recall}")
print(f"Short Precision: {short_precision}")
print(f"Short F1: {short_f1}")
print("=====================================")

print(f"Long Recall: {long_recall}")
print(f"Long Precision: {long_precision}")
print(f"Long F1: {long_f1}")
print("=====================================")

Recall: 0.4582659932659932
Precision: 0.45441863442709507
F1: 0.45632920485280953
Short Recall: 0.4209677419354839
Short Precision: 0.42419216192259934
Short F1: 0.4225688012224169
Long Recall: 0.6027967171717171
Long Precision: 0.5715462153820161
Long F1: 0.5867506654834951


## Haiku

Recall: 0.4582659932659932 \
Precision: 0.45441863442709507 \
F1: 0.45632920485280953

=====================================

Short Recall: 0.4209677419354839 \
Short Precision: 0.42419216192259934 \
Short F1: 0.4225688012224169

=====================================

Long Recall: 0.6027967171717171 \
Long Precision: 0.5715462153820161 \
Long F1: 0.5867506654834951

=====================================

## Llama 3 8b

## Llama 3 70b